In [1]:
import pandas as pd
import pyodbc
from os.path import join, normpath

downloads = 'C:/Users/balob/Downloads/DATA'

file_name = 'NEW_TARIFFS.csv'
file_name_res = 'NEW_TARIFFS_result.csv'

In [2]:
#Из базы данных OCS выгружаем TADIG и список дискаунтеров
 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT * FROM RDB_TADIG_CODES
'''
df_tadig = pd.read_sql_query(sql_srt, cnxn)
df_tadig['TADIG_CODE_ID']=df_tadig['TADIG_CODE_ID'].astype('int')
df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
display(df_tadig.head())

sql_srt=\
'''
SELECT * FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=323 AND [end_date] IS NULL AND [is_discounted]=1
'''
df_disc = pd.read_sql_query(sql_srt, cnxn)
display(df_disc.head())

cnxn.close()

,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE
0,987,4,AFGAW
1,988,5,AFGAR
2,989,6,AFG55
3,990,7,AFGTD
4,991,9,ALBAM


,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,NETWORK_ID,SPONSOR_ID,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,MT_SMS_RATE,DATA_RATE,CURRENCY_ID,START_DATE,END_DATE,is_discounted,HIDDEN
0,24331.0,323.0,366.0,1.0,0.600,0.1,0.0500,0.0,0.0440,1.0,2019-08-01,None,True,False
1,24353.0,323.0,1208.0,1.0,0.200,0.0,0.1200,0.0,0.0400,1.0,2019-01-01,None,True,False
2,24360.0,323.0,1036.0,2.0,0.048,0.0,0.0072,0.0,0.0072,1.0,2019-08-01,None,True,False
3,24132.0,323.0,224.0,2.0,0.096,0.0,0.0312,0.0,0.0144,1.0,2019-08-01,None,True,True
4,24133.0,323.0,412.0,2.0,0.960,0.0,0.3360,0.0,0.0960,1.0,2019-08-01,None,True,True


In [3]:
#Загружаем данные из csv файла с тарифами
#Пример файла NEW_TARIFFS.csv

df=pd.read_csv(join(downloads, file_name),sep=',')
df.head(3)

,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB
0,Albania,Vodafone,ALBVF,21.5,21.5,7.4,0.6,1,1
1,Anguilla,Cable and Wireless,AIACW,29.0,29.0,9.1,2.9,1,1
2,Antigua and Barbuda,C & W,ATGCW,29.0,29.0,9.1,2.9,1,1


In [4]:
#Объединяем тарифы с TADIG

df1=pd.merge(df,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
df1.head(3)

,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE
0,Albania,Vodafone,ALBVF,21.5,21.5,7.4,0.6,1,1,994,12,ALBVF
1,Anguilla,Cable and Wireless,AIACW,29.0,29.0,9.1,2.9,1,1,1000,20,AIACW
2,Antigua and Barbuda,C & W,ATGCW,29.0,29.0,9.1,2.9,1,1,1002,23,ATGCW


In [5]:
#Проверяем, что ничего не пропало

print(df.count(),'\n')
print(df1.count())

df1[df1.TADIG_CODE_ID.isnull()]

# df1[df1.TADIG=='FINTA']

Country                       278
Partner Name                  278
TADIG                         278
MO_CALL_RATE                  278
MT_CALL_RATE                  278
MO_SMS_RATE                   278
DATA_RATE                     278
OutgoingVoiceIncrement_SEC    278
DataIncrement_KB              278
dtype: int64 

Country                       278
Partner Name                  278
TADIG                         278
MO_CALL_RATE                  278
MT_CALL_RATE                  278
MO_SMS_RATE                   278
DATA_RATE                     278
OutgoingVoiceIncrement_SEC    278
DataIncrement_KB              278
TADIG_CODE_ID                 278
NETWORK_ID                    278
TADIG_CODE                    278
dtype: int64


,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE


In [6]:
# Проверить, все ли дискаунтеры. Для сверки берем один из тарифных планов.
# Данный шаг нужен для плано сторонних компаний, создаваемых на основе предоставленных ими данных. Например, Махком.

df2=pd.merge(df1,df_disc[['NETWORK_ID','is_discounted']],on='NETWORK_ID',how='left')
display(df2.head(3))
df2[df2.is_discounted.isnull()]

,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE,is_discounted
0,Albania,Vodafone,ALBVF,21.5,21.5,7.4,0.6,1,1,994,12,ALBVF,True
1,Albania,Vodafone,ALBVF,21.5,21.5,7.4,0.6,1,1,994,12,ALBVF,True
2,Anguilla,Cable and Wireless,AIACW,29.0,29.0,9.1,2.9,1,1,1000,20,AIACW,True


,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE,is_discounted
110,Cyprus,MTN Cyprus Limited,CYPK9,21.5,21.5,7.4,0.6,1,1,1828,230,CYPK9,NaN
111,Cyprus,MTN Cyprus Limited,CYPSC,21.5,21.5,7.4,0.6,1,1,1114,230,CYPSC,NaN
397,Philippines,Smart Communications Inc,PHLSR,31.8,31.8,10.3,1.9,1,1,1387,729,PHLSR,NaN


In [7]:
#Добавляем поля для удобства выгрузки
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

df2['ROAMING_PLAN_RULE_ID']=''
df2['ROAMING_PLAN_ID']=365
df2['SPONSOR_ID']='NULL'
df2['MT_SMS_RATE']=0
df2['CURRENCY_ID']=1 #USD
df2['START_DATE']='2019-08-01 00:00:00'
df2['END_DATE']='NULL'
#df2['is_discounted']=1
df2['is_discounted']=df2.is_discounted.fillna(value = 0).astype('int64')
df2.head()

,Country,Partner Name,TADIG,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,TADIG_CODE_ID,NETWORK_ID,TADIG_CODE,is_discounted,ROAMING_PLAN_RULE_ID,ROAMING_PLAN_ID,SPONSOR_ID,MT_SMS_RATE,CURRENCY_ID,START_DATE,END_DATE
0,Albania,Vodafone,ALBVF,21.5,21.5,7.4,0.6,1,1,994,12,ALBVF,1,,365,NULL,0,1,2019-08-01 00:00:00,NULL
1,Albania,Vodafone,ALBVF,21.5,21.5,7.4,0.6,1,1,994,12,ALBVF,1,,365,NULL,0,1,2019-08-01 00:00:00,NULL
2,Anguilla,Cable and Wireless,AIACW,29.0,29.0,9.1,2.9,1,1,1000,20,AIACW,1,,365,NULL,0,1,2019-08-01 00:00:00,NULL
3,Anguilla,Cable and Wireless,AIACW,29.0,29.0,9.1,2.9,1,1,1000,20,AIACW,1,,365,NULL,0,1,2019-08-01 00:00:00,NULL
4,Antigua and Barbuda,C & W,ATGCW,29.0,29.0,9.1,2.9,1,1,1002,23,ATGCW,1,,365,NULL,0,1,2019-08-01 00:00:00,NULL


In [8]:
#Выгружаем без дубликатов в csv

df2[['ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MO_CALL_RATE',
'MT_CALL_RATE',
'MO_SMS_RATE',
'MT_SMS_RATE',
'DATA_RATE',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',
'OutgoingVoiceIncrement_SEC',
'DataIncrement_KB']].drop_duplicates().to_csv(join(downloads, file_name_res),index=False)

-----------------------------------------------------------------------------------------
Далее используем INSERT_ROAMING_PLAN_RULES_ХХХХХХ.xlsx